In [1]:
import os
import datetime

In [28]:
class Downloader:
    def __init__(self):
        self.tar_path = '/home/ubuntu/synthetic/tar'
        self.bin_path = '/mnt/hgfs/kagoshima/bin'  # Seminarをマウントしている
        %mkdir -p $self.tar_path
        self.delete_directory(tar_flag=True)
        %mkdir -p $self.tar_path
        %mkdir -p $self.bin_path
        %cd ~/synthetic/tar
    
    def set_date(self, dt):
        # URLを指定
        directory = 'http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/jma-radar/synthetic/original'
        date = dt.strftime('%Y/%m/%d')
        self.timestamp = dt.strftime('%Y%m%d%H%M%S')
        self.tar_filename = 'Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV__grib2.tar'
        self.URL = directory +'/'+ date +'/'+ self.tar_filename
        self.bin_path_year = self.bin_path + '/' + dt.strftime('%Y')
        %mkdir -p $self.bin_path_year
        
    def get_bin_file(self):
        # binファイルがあるかを確認
        bin_file_path = self.bin_path_year+'/Z__C_RJTD_'+self.timestamp+'_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin'
        if os.path.exists(bin_file_path):
            #print('already exist: ',bin_file_path)
            pass
        else:
            # tarファイルを~/tarに保存
            wget_result = !wget -nc -P $self.tar_path $self.URL
            download_error = [s for s in wget_result if 'エラー' in s]
            if download_error:  # ダウンロードエラーの場合
                print('download error: '+self.URL)
                with open('/mnt/hgfs/kagoshima/error_URL.txt', 'a') as f:
                    f.write(self.URL+'\n')
            else:
                # tarファイルから1kmメッシュデータのみを取り出す
                filelist = !tar -tf $self.tar_filename  # tarファイル内ファイル名リスト
                ggis_name = [s for s in filelist if 'Ggis1km' in s][0]  # ファイル名を取得
                !tar -C $self.bin_path_year -xvf $self.tar_filename $ggis_name # tarファイルから1kmメッシュ全国合成レーダーエコー強度GPVのみを/binに取り出す
                !rm $self.tar_filename  # tarファイルは不要なので削除
    
    def delete_directory(self, tar_flag=False, bin_flag=False):
        if tar_flag:
            !rm -r $self.tar_path
        if bin_flag:
            !rm -r $self.bin_path

In [27]:
start = '2008/01/07 15:00:00'
end = '2018/01/01 00:00:00'
dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')

downloader = Downloader()
while dt < dt_limit:
    downloader.set_date(dt)
    downloader.get_bin_file()
    dt = dt + datetime.timedelta(minutes=10)

sh: 0: getcwd() failed: No such file or directory
sh: 0: getcwd() failed: No such file or directory
/home/ubuntu/synthetic/tar
Z__C_RJTD_20080108054000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108055000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108060000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108061000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108062000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108063000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108064000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108065000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108070000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108071000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108072000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108073000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z__C_RJTD_20080108074000_RDR_JMAGPV_Ggis1km_Prr10lv_ANAL_grib2.bin
Z_

PermissionError: [Errno 13] Permission denied: '/mnt/hgfs/kagoshima/error_URL.txt'